# MetaInfoReader - e-Stat統計メタ情報取得ガイド

このノートブックでは、jpy-datareaderライブラリの`MetaInfoReader`を使用して、日本の政府統計e-Statから統計表のメタ情報（項目定義、分類コード等）を取得する方法を学習します。

## 概要

`MetaInfoReader`は、e-Stat APIの「メタ情報取得」機能を使用して、以下の情報を取得できます：
- 統計表の項目分類情報（表章項目、分類事項、地域、時間軸等）
- 各分類のコード・名称対応表
- 階層構造を持つ分類の階層情報
- 統計表の基本情報（タイトル、調査概要等）

## 事前準備

1. **統計表IDの特定**
   - `StatsListReader`で事前に統計表IDを取得
   - 対象となる統計表の`statsDataId`が必要

2. **必要なライブラリ**
   - `jpy-datareader`
   - `pandas`
   - `python-dotenv` (環境変数管理用)



## 1. ライブラリのインポートと設定

In [ ]:
import os
import sys
from pathlib import Path
import requests
import pandas as pd
from dotenv import load_dotenv

# 日本語表示の設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 60)

## 2. MetaInfoReaderインポート

In [ ]:
from jpy_datareader.estat import MetaInfoReader

## 3. APIキーの読み込み

In [ ]:
# APIキーの読み込み
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")

if not appId:
    print("⚠️ APIキーが設定されていません")
    print("以下の環境変数のいずれかを設定してください:")
    print("- ESTAT_APP_ID")
    print("- E_STAT_APPLICATION_ID")
    print("- ESTAT_APPLICATION_ID")
    # 直接設定する場合（テスト用）
    # appId = "あなたのAPIキーをここに入力"
else:
    print(f"✅ APIキーが正常に読み込まれました: {appId[:10]}...")

## 基本的な使用方法

### 4. 統計表IDの設定と基本情報取得

In [ ]:
# 国勢調査の統計表IDを使用（例）
statsDataId = "0002070010"  # 国勢調査：男女別人口及び世帯の種類別世帯数

print(f"=== 統計表ID: {statsDataId} のメタ情報取得 ===")

# MetaInfoReaderインスタンスの作成
metainfo = MetaInfoReader(
    api_key=appId, 
    statsDataId=statsDataId,
    has_lv_hierarchy=True,  # 階層情報も取得
    explanationGetFlg="Y"   # 説明情報も取得
)

print("MetaInfoReaderインスタンスを作成しました")
print(f"統計表ID: {metainfo.statsDataId}")
print(f"階層処理: {metainfo.has_lv_hierarchy}")

### 5. JSON形式での生データ取得

In [ ]:
# JSON形式で生データを取得
print("=== JSON形式でのメタ情報取得 ===")
meta_json = metainfo.read_json()

# APIレスポンスの構造を確認
print("APIレスポンスの主要キー:")
for key in meta_json.keys():
    print(f"- {key}")

# メタ情報の基本構造を確認
get_meta_info = meta_json.get('GET_META_INFO', {})
print(f"\nGET_META_INFOの主要キー:")
for key in get_meta_info.keys():
    print(f"- {key}")

# ステータス情報を確認
result_info = get_meta_info.get('RESULT', {})
print(f"\nAPI実行結果:")
print(f"- ステータス: {result_info.get('STATUS')}")
print(f"- 実行日時: {result_info.get('DATE')}")

# 統計表基本情報を確認
table_inf = get_meta_info.get('METADATA_INF', {}).get('TABLE_INF', {})
print(f"\n=== 統計表基本情報 ===")
print(f"統計名: {table_inf.get('STAT_NAME')}")
print(f"表題: {table_inf.get('TITLE')}")
print(f"調査年月: {table_inf.get('SURVEY_DATE')}")
print(f"公開年月: {table_inf.get('OPEN_DATE')}")

### 6. 分類オブジェクト情報の取得

In [ ]:
# 分類オブジェクト（CLASS_OBJ）の詳細分析
print("=== 分類オブジェクトの詳細情報 ===")

class_objs = get_meta_info.get('METADATA_INF', {}).get('CLASS_INF', {}).get('CLASS_OBJ', [])
print(f"分類オブジェクト数: {len(class_objs)}")

print("\n=== 各分類オブジェクトの概要 ===")
for i, class_obj in enumerate(class_objs):
    class_id = class_obj.get('@id')
    class_name = class_obj.get('@name')
    class_data = class_obj.get('CLASS', [])
    
    # CLASSデータの件数を確認
    if isinstance(class_data, list):
        class_count = len(class_data)
    elif isinstance(class_data, dict):
        class_count = 1
    else:
        class_count = 0
    
    print(f"{i}. ID: {class_id}")
    print(f"   名称: {class_name}")
    print(f"   項目数: {class_count}件")
    
    # サンプルデータを表示
    if class_data and isinstance(class_data, list) and len(class_data) > 0:
        sample = class_data[0]
        print(f"   サンプルコード: {sample.get('@code', 'N/A')}")
        print(f"   サンプル名称: {sample.get('@name', 'N/A')}")
        if '@level' in sample:
            print(f"   階層レベル: {sample.get('@level', 'N/A')}")
    print()

## 実践的なデータ処理

### 7. 分類データの構造化取得

In [ ]:
# read_class_objs()メソッドで構造化されたデータを取得
print("=== 構造化された分類データの取得 ===")
meta_classes = metainfo.read_class_objs()

print(f"取得した分類数: {len(meta_classes)}件")

print("\n=== 各分類の詳細情報 ===")
for i, class_data in enumerate(meta_classes):
    class_id = class_data['id']
    class_name = class_data['name']
    df = class_data['meta_dataframe']
    
    print(f"{i}. 分類ID: {class_id}")
    print(f"   分類名: {class_name}")
    print(f"   データ形状: {df.shape}")
    print(f"   カラム: {list(df.columns)}")
    
    # 階層データがある場合
    if 'hierarchy' in class_data:
        hierarchy_df = class_data['hierarchy']
        print(f"   階層データ形状: {hierarchy_df.shape}")
        print(f"   階層カラム: {list(hierarchy_df.columns)}")
    
    print()

### 8. 最大の分類データを取得（デフォルト動作）

In [ ]:
# read()メソッドで最も項目数の多い分類データを取得
print("=== 最大分類データの取得 ===")
largest_df = metainfo.read()

print(f"取得データ形状: {largest_df.shape}")
print(f"カラム一覧:")
for i, col in enumerate(largest_df.columns):
    print(f"  {i+1}. {col}")

if len(largest_df) > 0:
    print(f"\n=== データサンプル（最初の5件）===")
    print(largest_df.head().to_string(index=False))
    
    print(f"\n=== データ統計情報 ===")
    print(f"総項目数: {len(largest_df)}件")
    
    # 階層レベルの分布（存在する場合）
    level_columns = [col for col in largest_df.columns if 'レベル' in col or 'level' in col.lower()]
    if level_columns:
        level_col = level_columns[0]
        level_dist = largest_df[level_col].value_counts().sort_index()
        print(f"\n階層レベル分布:")
        for level, count in level_dist.items():
            print(f"  レベル{level}: {count}件")

## 高度な使用例

### 9. 特定分類の詳細分析

In [ ]:
# 地域分類（通常はarea）の詳細分析
print("=== 地域分類の詳細分析 ===")

# 地域分類を探す
area_class = None
for class_data in meta_classes:
    if class_data['id'] == 'area' or '地域' in class_data['name']:
        area_class = class_data
        break

if area_class:
    area_df = area_class['meta_dataframe']
    print(f"地域分類名: {area_class['name']}")
    print(f"地域数: {len(area_df)}件")
    
    # 都道府県レベルのデータを抽出（レベル2の場合が多い）
    level_columns = [col for col in area_df.columns if 'レベル' in col or 'level' in col.lower()]
    if level_columns:
        level_col = level_columns[0]
        prefectures = area_df[area_df[level_col] == 2]  # レベル2が都道府県の場合
        
        print(f"\n都道府県データ（レベル2）: {len(prefectures)}件")
        if len(prefectures) > 0:
            print("都道府県サンプル:")
            code_col = [col for col in prefectures.columns if 'コード' in col][0]
            name_col = [col for col in prefectures.columns if '名称' in col or col == area_class['name']][0]
            
            for _, row in prefectures.head(10).iterrows():
                print(f"  {row[code_col]}: {row[name_col]}")
    
    # 階層データがある場合の分析
    if 'hierarchy' in area_class:
        hierarchy_df = area_class['hierarchy']
        print(f"\n階層データ分析:")
        print(f"階層データ形状: {hierarchy_df.shape}")
        
        # 階層レベル別の表示
        hierarchy_cols = [col for col in hierarchy_df.columns if '階層' in col]
        if hierarchy_cols:
            print(f"\n階層構造サンプル（最初の5件）:")
            print(hierarchy_df[hierarchy_cols[:3]].head().to_string(index=False))
else:
    print("地域分類が見つかりませんでした")

### 10. 時間軸分類の分析

In [ ]:
# 時間軸分類の詳細分析
print("=== 時間軸分類の詳細分析 ===")

# 時間軸分類を探す
time_class = None
for class_data in meta_classes:
    if class_data['id'] == 'time' or '時間' in class_data['name'] or '年次' in class_data['name']:
        time_class = class_data
        break

if time_class:
    time_df = time_class['meta_dataframe']
    print(f"時間軸分類名: {time_class['name']}")
    print(f"時系列数: {len(time_df)}件")
    
    # 時系列データの概要
    code_col = [col for col in time_df.columns if 'コード' in col][0]
    name_col = [col for col in time_df.columns if '名称' in col or col == time_class['name']][0]
    
    print(f"\n時系列範囲:")
    print(f"最初: {time_df.iloc[0][code_col]} - {time_df.iloc[0][name_col]}")
    print(f"最後: {time_df.iloc[-1][code_col]} - {time_df.iloc[-1][name_col]}")
    
    print(f"\n時系列サンプル（最初の10件）:")
    for _, row in time_df.head(10).iterrows():
        print(f"  {row[code_col]}: {row[name_col]}")
        
    print(f"\n時系列サンプル（最後の5件）:")
    for _, row in time_df.tail(5).iterrows():
        print(f"  {row[code_col]}: {row[name_col]}")
else:
    print("時間軸分類が見つかりませんでした")

### 11. 表章項目分類の分析

In [ ]:
# 表章項目分類の詳細分析
print("=== 表章項目分類の詳細分析 ===")

# 表章項目分類を探す
tab_class = None
for class_data in meta_classes:
    if class_data['id'] == 'tab' or '表章' in class_data['name'] or '項目' in class_data['name']:
        tab_class = class_data
        break

if tab_class:
    tab_df = tab_class['meta_dataframe']
    print(f"表章項目分類名: {tab_class['name']}")
    print(f"表章項目数: {len(tab_df)}件")
    
    # 表章項目の一覧表示
    code_col = [col for col in tab_df.columns if 'コード' in col][0]
    name_col = [col for col in tab_df.columns if '名称' in col or col == tab_class['name']][0]
    unit_cols = [col for col in tab_df.columns if '単位' in col]
    
    print(f"\n=== 表章項目一覧 ===")
    for _, row in tab_df.iterrows():
        item_info = f"{row[code_col]}: {row[name_col]}"
        if unit_cols:
            unit = row[unit_cols[0]]
            if pd.notna(unit) and str(unit).strip():
                item_info += f" ({unit})"
        print(f"  {item_info}")
        
    # 単位の分布
    if unit_cols:
        unit_col = unit_cols[0]
        unit_dist = tab_df[unit_col].value_counts()
        print(f"\n単位別項目数:")
        for unit, count in unit_dist.items():
            if pd.notna(unit) and str(unit).strip():
                print(f"  {unit}: {count}件")
else:
    print("表章項目分類が見つかりませんでした")

### 12. 階層構造の詳細分析

In [ ]:
# 階層構造を持つ分類の詳細分析
print("=== 階層構造の詳細分析 ===")

hierarchy_classes = [cls for cls in meta_classes if 'hierarchy' in cls]

if hierarchy_classes:
    print(f"階層構造を持つ分類数: {len(hierarchy_classes)}件")
    
    for i, class_data in enumerate(hierarchy_classes):
        class_name = class_data['name']
        hierarchy_df = class_data['hierarchy']
        
        print(f"\n{i+1}. 分類名: {class_name}")
        print(f"   階層データ形状: {hierarchy_df.shape}")
        
        # 階層レベルの確認
        level_cols = [col for col in hierarchy_df.columns if 'level' in col]
        hierarchy_cols = [col for col in hierarchy_df.columns if '階層' in col]
        
        if hierarchy_cols:
            max_level = len([col for col in hierarchy_cols if col.startswith(class_name)])
            print(f"   最大階層レベル: {max_level}")
            
            # 各階層の項目数
            print(f"   階層別ユニーク項目数:")
            for level in range(1, max_level + 1):
                hierarchy_col = f"{class_name}階層{level}"
                if hierarchy_col in hierarchy_df.columns:
                    unique_count = hierarchy_df[hierarchy_col].nunique()
                    print(f"     レベル{level}: {unique_count}項目")
            
            # サンプル階層構造の表示
            print(f"\n   階層構造サンプル（最初の5件）:")
            sample_cols = hierarchy_cols[:min(3, len(hierarchy_cols))]
            sample_df = hierarchy_df[sample_cols].head()
            for _, row in sample_df.iterrows():
                hierarchy_path = " > ".join([str(val) for val in row if pd.notna(val)])
                print(f"     {hierarchy_path}")
else:
    print("階層構造を持つ分類が見つかりませんでした")

## データの保存と活用

### 13. メタ情報の保存

In [ ]:
# メタ情報をファイルに保存
print("=== メタ情報の保存 ===")

# 統計表基本情報の保存
table_info = {
    'statsDataId': statsDataId,
    'STAT_NAME': metainfo.STAT_NAME,
    'TITLE': metainfo.TITLE,
    'SURVEY_DATE': metainfo.SURVEY_DATE,
    'OPEN_DATE': metainfo.OPEN_DATE,
    'UPDATED_DATE': metainfo.UPDATED_DATE,
    'MAIN_CATEGORY': metainfo.MAIN_CATEGORY,
    'SUB_CATEGORY': metainfo.SUB_CATEGORY
}

table_info_df = pd.DataFrame([table_info])
table_info_file = f"統計表情報_{statsDataId}.csv"
table_info_df.to_csv(table_info_file, index=False, encoding='utf-8-sig')
print(f"✅ 統計表基本情報を {table_info_file} に保存しました")

# 各分類データの保存
saved_files = []
for class_data in meta_classes:
    class_id = class_data['id']
    class_name = class_data['name']
    df = class_data['meta_dataframe']
    
    # ファイル名の作成（特殊文字を除去）
    safe_name = "".join(c for c in class_name if c.isalnum() or c in (' ', '-', '_')).rstrip()
    filename = f"分類_{class_id}_{safe_name}_{statsDataId}.csv"
    
    # データの保存
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    saved_files.append(filename)
    print(f"✅ {class_name}（{class_id}）データを {filename} に保存しました")
    
    # 階層データがある場合も保存
    if 'hierarchy' in class_data:
        hierarchy_df = class_data['hierarchy']
        hierarchy_filename = f"階層_{class_id}_{safe_name}_{statsDataId}.csv"
        hierarchy_df.to_csv(hierarchy_filename, index=False, encoding='utf-8-sig')
        saved_files.append(hierarchy_filename)
        print(f"✅ {class_name}階層データを {hierarchy_filename} に保存しました")

print(f"\n合計 {len(saved_files)} ファイルを保存しました")

### 14. 次のステップ - データ取得への準備

In [ ]:
# メタ情報を活用したデータ取得の準備
print("=== 次のステップ: データ取得への準備 ===")

print(f"統計表ID: {statsDataId}")
print(f"統計名: {metainfo.STAT_NAME}")
print(f"表題: {metainfo.TITLE}")

print(f"\n=== データ取得で使用できる分類コード ===")
for class_data in meta_classes:
    class_id = class_data['id']
    class_name = class_data['name']
    df = class_data['meta_dataframe']
    
    print(f"\n{class_id} ({class_name}):")
    
    # コードカラムを特定
    code_cols = [col for col in df.columns if 'コード' in col]
    if code_cols:
        code_col = code_cols[0]
        codes = df[code_col].tolist()
        
        # 最初の5つのコードを表示
        sample_codes = codes[:5]
        print(f"  サンプルコード: {', '.join(map(str, sample_codes))}")
        print(f"  総コード数: {len(codes)}件")
        
        # 特定コードの使用例
        if len(codes) > 0:
            example_code = codes[0]
            print(f"  使用例: {class_id}={example_code}")

print(f"\n=== StatsDataReaderでのデータ取得例 ===")
print(f"""
from jpy_datareader.estat import StatsDataReader

# 基本的なデータ取得
data_reader = StatsDataReader(
    api_key=appId,
    statsDataId="{statsDataId}"
)
df_data = data_reader.read()

# 特定条件でのデータ取得例
data_reader_filtered = StatsDataReader(
    api_key=appId,
    statsDataId="{statsDataId}",
    # 必要に応じて分類コードで絞り込み
    # area="01000",  # 北海道のデータのみ
    # time="2020",   # 2020年のデータのみ
)
df_filtered = data_reader_filtered.read()
""")

## まとめ

### MetaInfoReaderの主要な機能

1. **基本情報取得**: 統計表の概要、タイトル、調査概要
2. **分類情報取得**: 
   - `read_class_objs()`: 全分類の構造化データ
   - `read()`: 最大の分類データ（時間軸以外）
   - `read_json()`: 生JSONデータ
3. **階層処理**: `has_lv_hierarchy=True`で階層構造の展開
4. **言語選択**: `lang="E"`で英語表記の取得

### 活用のポイント

- **データ取得前の必須ステップ**: 統計データの取得前に分類コードと構造を把握
- **効率的な絞り込み**: 必要な分類コードを特定してデータ取得を最適化
- **階層構造の理解**: 地域や産業分類などの階層関係を正確に把握
- **メタデータの保存**: 後続の分析で参照できるよう分類情報を保存

### 次のステップ

メタ情報を把握したら、`StatsDataReader`を使用して実際の統計データを効率的に取得することができます。分類コードを適切に指定することで、必要なデータのみを絞り込んで取得可能です。